## 2. Heat pipe design

A heated fluid at temperature $T$ (degrees above ambient temperature) flows in a pipe with fixed length and circular cross section with radius $r$. A layer of insulation, with thickness $w$, surrounds the pipe to reduce heat loss through the pipe walls ($w$ is much smaller than $r$). The design variables in this problem are $T$, $r$, and $w$.

The energy cost due to heat loss is roughly equal to $\alpha_1 Tr/w$. The cost of the pipe, which has a fixed wall thickness, is approximately proportional to the total material, i.e., it is given by $\alpha_2 r$. The cost of the insulation is also approximately proportional to the total insulation material, i.e., roughly $\alpha_3 r w$. The total cost is the sum of these three costs.

The heat flow down the pipe is entirely due to the flow of the fluid, which has a fixed
velocity, i.e., it is given by $\alpha_4 Tr^2$. The constants $\alpha_i$ are all positive, as are the variables $T$, $r$, and $w$.

Now the problem: maximize the total heat flow down the pipe, subject to an upper limit
$C_\text{max}$ on total cost, and the constraints
$$
T_\text{min} \le T \le T_\text{max},
\qquad
r_\text{min} \le r \le r_\text{max}
\qquad
w_\text{min} \le w \le w_\text{max},
\qquad
w \le 0.1 r
$$

**(a)** Express this problem as a geometric program, and convert it into a convex optimization problem.

**(b)** Consider a simple instance of this problem, where $C_\text{max} = 500$ and $\alpha_1=\alpha_2=\alpha_3=\alpha_4 = 1$. Also assume for simplicity that each variable has a lower bound of zero and no upper bound. Solve this problem using JuMP. 
Use the `Ipopt` solver and the command `@addNLConstraint(...)` to specify nonlinear constraints such as log-sum-exp functions. What is the optimal $T$, $r$, and $w$?

###### Solution to problem 1(a)

###### Geometric Program
$$\mbox{maximize} \ \ \ \ \alpha_4Tr^2$$

$$\mbox{subject to} \ \ \ \alpha_1Trw^{-1} + \alpha_2r + \alpha_3rw \leq C_{max}$$
$$T_{min} \leq T \leq T_{max}$$
$$r_{min} \leq r \leq r_{max}$$
$$w_{min} \leq w \leq w_{max}$$
$$w \leq 0.1r$$

Note the maximization can be easily transformed to minimization over its inverse.

###### Convex Optimization problem
Let $x = \log T, y = \log r, z = \log w$, then we get
$$\mbox{maximize} \ \ \ \ \ x + 2y$$
$$\mbox{subject to} \ \ \ \ \log{(e^{\log \alpha_1 + x + y - z} + e^{\log \alpha_2 + y} + e^{\log \alpha_3 + y + z})} \leq \log C_{max}$$
$$\log T_{min} \leq x \leq \log T_{max}$$
$$\log r_{min} \leq y \leq \log r_{max}$$
$$\log w_{min} \leq z \leq \log w_{max}$$
$$z \leq y + \log0.1$$

In [10]:
# Solution to problem 1(b)
using Ipopt
α1 = α2 = α3 = α4 = 1
Cmax = 500

using JuMP, Ipopt
m = Model(Ipopt.Optimizer)

@variable(m, x)
@variable(m, y)
@variable(m, z)

#add Nonlinear Constraint
@NLconstraint(m, log(exp(x+y-z+log(α1)) + exp(y+log(α2)) + exp(y+z+log(α3))) <= log(Cmax))

@constraint(m, z <= y + log(0.1))
@objective(m, Max, x+2y)
@time(optimize!(m))

println("T: ", exp(JuMP.value(x)))
println("r: ", exp(JuMP.value(y)))
println("w: ", exp(JuMP.value(z)))
println("Max Heat Flow: ", exp(JuMP.objective_value(m)))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.30e+00 7.71e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [11]:
# Bonus --- the nonlinear solver should be able to solve the non-convex problem directly too!
# NOTE: this is not an acceptable solution; the point of the exercise was to convert to a convex program.

m = Model(Ipopt.Optimizer)
@variable(m, T >= 0)
@variable(m, r >= 0)
@variable(m, w >= 0)

@NLconstraint(m, α1*T*r/w + α2*r + α3*r*w <= Cmax)
@constraint(m, w <= 0.1*r)
@NLobjective(m, Max, α4*T*r^2)
@time(optimize!(m))

println("T: ", JuMP.value(T))
println("r: ", JuMP.value(r))
println("w: ", JuMP.value(w))
println("Max Heat Flow: ", JuMP.objective_value(m))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999700e-07 9.00e-03 3.43e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [12]:
using JuMP, Ipopt
m= Model(Ipopt.Optimizer)
α1 = 1
α2 = 1
α3 = 1
α4 = 1
Cmax = 500
# x = log(T) # y = log(r) # z = log(w)
@variable(m, x)
@variable(m, y)
@variable(m, z)
#each variable has a lower bound of zero and no upper bound
@constraint(m, x >= 0)
@constraint(m, y >= 0)
@constraint(m, z >= 0)
#nonlinear constraints
@NLconstraint(m, log((exp(log(α1/Cmax)+x+y-z)) + (exp(log(α2/Cmax)+y)) + (exp(
log(α3/Cmax)+y+z))) <= 0)
@NLconstraint(m, log(exp(log(10)+z-y)) <=0)
@NLobjective(m, Max, log(α4)+x+2*y)

optimize!(m)
T = exp(JuMP.value(x))
r = exp(JuMP.value(y))
w = exp(JuMP.value(z))
println("Optimal T is: ", T)
println("Optimal r is: ", r)
println("Optimal w is: ", w)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.30e+00 9.81e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

Optimal T is: 23.840239119135667
Optimal r is: 46.390428373110055
Optimal w is: 4.639042761905939
